In [1]:
import pandas as pd
import numpy as np


In [37]:
file_1 = pd.read_csv('P031749_TNLCF_Community_Research_England_wtd_Tables_20231114_Private.csv')
file_1.head()

,Unnamed: 0,<< Contents,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57
0,NaN,National Lottery Community Fund - CRI - Wave 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Table 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Q1_W3. Do you feel like you are a part of your...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,BASE: All respondents,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Gender,NaN,Age,NaN,NaN,NaN,NaN,...,NaN,Intention to volunteer in 2023,NaN,NaN,NaN,NaN,Strength of local community spirit,NaN,Worried about the impact of climate change on ...,NaN


## Functions

In [74]:
def get_subject_and_question_into_json(file):
    subject = file['<< Contents'][0]
    question = file['<< Contents'][2]
    json_file = {}
    json_file['subject'] = subject
    json_file['question'] = question
    return json_file

def fill_with_preceding(row):
    for i in range(1, len(row)):
        if pd.isna(row[i]):
            row[i] = row[i-1]
    return row



def get_json(file):
    end_row = file[file['<< Contents'].str.contains('Columns tested', case=False, na=False)].index[0]
    new_df = file.iloc[4:end_row]
    to_exclude = ['Unnamed: 0','<< Contents','Unnamed: 2']
    file_curated = new_df[[col for col in new_df.columns if col not in to_exclude]]
    file_curated = file_curated.apply(fill_with_preceding, axis=1)
    file_curated.columns = pd.MultiIndex.from_arrays([file_curated.iloc[0], file_curated.iloc[1]])
    file_curated.columns = [f'{i}_{j}' for i,j in file_curated.columns]
    file_curated.reset_index(drop=True, inplace=True)
    file_curated.drop([0,1], inplace=True)
    return file_curated.to_json(orient='records')

def compile_json(file):
    json_file = get_subject_and_question_into_json(file)
    json_file['data'] = get_json(file)
    return json_file

## Tests

In [67]:
metadata = get_subject_and_question_into_json(file_1)

In [68]:
table_content = get_json(file_1)

In [72]:
metadata['content'] = table_content

In [76]:
## Run this to extract data from csv file
compile_json(file_1)

{'subject': 'National Lottery Community Fund - CRI - Wave 3',
 'question': "Q1_W3. Do you feel like you are a part of your local community? By 'local community', we mean the local area where you live.",
 'data': '[{"Gender_Male":"a","Gender_Female":"b","Age_16-24":"a","Age_25-34":"b","Age_35-44":"c","Age_45-54":"d","Age_55-64":"e","Age_65+":"f","Age_18-34":"g","Age_35-54":"h","Age_55+":"i","English region_North-West":"a","English region_North-East":"b","English region_Yorkshire & Humberside":"c","English region_West Midlands":"d","English region_East Midlands":"e","English region_South-West":"f","English region_South-East":"g","English region_Eastern":"h","English region_London":"i","IMD ranking_Deciles 1-3":"a","IMD ranking_Deciles 4-10":"b","Social grade_AB":"a","Social grade_C1":"b","Social grade_C2":"c","Social grade_D":"d","Social grade_E":"e","Social grade_ABC1":"f","Social grade_C2DE":"g","Education_Non-graduates":"a","Education_Graduates":"b","Ethnicity_White":"a","Ethnicity_Mi